In [1]:
import cv2
import torch
import numpy as np
from moviepy.editor import VideoFileClip

In [2]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\Rabeea/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-25 Python-3.9.16 torch-1.11.0+cpu CPU



  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [11]:
# Configuration
line_position = 1000
current_detections = []
car_count = 0
car_ids = []
current_id = 0

def get_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

def get_distance(pt1, pt2):
    return np.sqrt((pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2)

In [18]:
# Video source input
cap = cv2.VideoCapture(r'D:\Github Files\2023-Learning-Journey\Projects\Yolov5 Model Car Counting\Cars On Highway.mp4')

In [19]:
# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r'D:\Github Files\2023-Learning-Journey\Projects\Yolov5 Model Car Counting\processed_video', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

In [10]:
while True:
    ret, frame = cap.read()        
    new_detections = []

    if not ret:
        break

    # Use YOLOv5 to detect vehicles
    results = model(frame)
    current_centroids = []
    
    for *xyxy, conf, cls in results.pred[0]:
        if int(cls) == 2:  # Assuming class ID for cars is 2
            x1, y1, x2, y2 = map(int, xyxy)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            center = get_center(x1, y1, x2-x1, y2-y1)
            current_centroids.append(center)

    # Match current centroids with previous detections
    for center in current_centroids:
        if len(current_detections) == 0:  # if it's the first detection
            current_id += 1
            new_detections.append((center, current_id))
        else:
            # Find the closest detected centroid from the previous frame
            distances = [get_distance(center, prev_center) for prev_center, _ in current_detections]
            min_distance = min(distances)
            index = distances.index(min_distance)
            
            if min_distance < 50:  # Threshold distance, you might need to adjust this
                new_detections.append((center, current_detections[index][1]))
            else:
                current_id += 1
                new_detections.append((center, current_id))

    # Counting cars based on line crossing
    for (x, y), car_id in new_detections:
        if (y < (line_position + 6)) and (y > (line_position - 6)):
            if car_id not in car_ids:
                car_ids.append(car_id)
                car_count += 1
                print("No. of cars detected : " + str(car_count))
                
        # Display ID near the centroid
        cv2.putText(frame, f"ID:{car_id}", (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Drawing the counting line
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 0, 255), 2)
    cv2.putText(frame, "VEHICLE COUNT : "+str(car_count), (320, 70), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)
    # cv2.imshow("Vehicle Count", frame)

    # Update current_detections with new_detections for the next frame
    current_detections = new_detections

    # Save the processed frame
    out.write(frame)  
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
#Display the saved video using moviepy
clip = VideoFileClip(r'D:\Github Files\2023-Learning-Journey\Projects\Yolov5 Model Car Counting\processed_video.mp4')
clip.ipython_display(width = 1080, maxduration = 200)